In [3]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['자연어 기반 SQL']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess
db_dict = dict()


#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(reversed(sorted(source_data_dir_split.iterdir())), desc=split):
        source_data = source_data.open()
        prev_db_id = None
        for line in source_data.readlines():
            line = json.loads(line)
            db_id = line.pop('db_id')
            #### data preprocess
            if 'source' in line:    ## _S
                db_dict[db_id] = line
                continue
            #### data preprocess end 
            
            if prev_db_id != db_id:
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                prev_db_id = db_id
                #### 자연어 기반 SQL
                data = {'input': {}, 'output': None}
                ## preprocess data from line
                data['input']['db_info'] = db_dict[db_id]
                data['input']['utterance'] = []
                data['output'] = []
            
            
            data['input']['utterance'].append(line['utterance'])
            data['output'].append(line['query'])
            
            
            ## preprocess data from line end
            # task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 자연어 기반 SQL end
            
            
            
            # break
        # break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

train: 2it [00:00,  2.61it/s]
valid: 2it [00:00, 22.41it/s]
